In [13]:
import numpy as np
import pandas as pd
from glob import glob
import xml.etree.ElementTree as xet
import cv2
import os
from shutil import copy

In [2]:
df = pd.read_csv("dataset/label.csv")
df.head()

,filepath,xmin,xmax,ymin,ymax
0,dataset/data\N1.xml,1093,1396,645,727
1,dataset/data\N100.xml,134,301,312,350
2,dataset/data\N101.xml,31,139,128,161
3,dataset/data\N102.xml,164,316,216,243
4,dataset/data\N103.xml,813,1067,665,724


## Labels YOLO

* center_X
* center_y
* width
* height

## Parsing

In [3]:
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find("filename").text
    filename = f"dataset/data/{name}"

    # width and height
    parser_size = parser.find("size")
    width = int(parser_size.find("width").text)
    height = int(parser_size.find("height").text)
    
    return filename, width, height

In [4]:
df[["filename", "width", "height"]] = df["filepath"].apply(parsing).apply(pd.Series)

df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,dataset/data\N1.xml,1093,1396,645,727,dataset/data/N1.jpeg,1920,1080
1,dataset/data\N100.xml,134,301,312,350,dataset/data/N100.jpeg,450,417
2,dataset/data\N101.xml,31,139,128,161,dataset/data/N101.jpeg,249,239
3,dataset/data\N102.xml,164,316,216,243,dataset/data/N102.jpeg,478,395
4,dataset/data\N103.xml,813,1067,665,724,dataset/data/N103.jpeg,1800,1200


### center_X, center_Y, width, height

In [5]:
df["center_x"] = (df["xmax"] + df["xmin"]) / (2 * df["width"])
df["center_y"] = (df["ymax"] + df["ymin"]) / (2 * df["height"])

df["bb_width"] = (df["xmax"] - df["xmin"]) / df["width"]
df["bb_height"] = (df["ymax"] - df["ymin"]) / df["height"]

df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,dataset/data\N1.xml,1093,1396,645,727,dataset/data/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,dataset/data\N100.xml,134,301,312,350,dataset/data/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,dataset/data\N101.xml,31,139,128,161,dataset/data/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,dataset/data\N102.xml,164,316,216,243,dataset/data/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,dataset/data\N103.xml,813,1067,665,724,dataset/data/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


## Split the Data into Train and Test

In [8]:
df_train = df.iloc[:200]
df_test = df.iloc[200:]

text.file

    class_id, center_x, center_y, bb_width, bb_height

In [14]:
train_folder = "data_images/train"

values = df_train[["filename", "center_x", "center_y", "bb_width", "bb_height"]].values
for fname,x,y,w,h in values:
    image_name = os.path.split(fname)[-1] # it will give N1.jpeg etc.
    txt_name = os.path.splitext(image_name)[0] # it will give only N1 (image name without any format)
    
    dst_image_path = os.path.join(train_folder, image_name) # destination name for copy process
    dst_label_file = os.path.join(train_folder, txt_name+".txt")
    
    # copy each image into the folder
    copy(fname, dst_image_path)

    # generate .txt file which has label informatation.
    label_txt = f"0 {x} {y} {w} {h}" # -> class id will be 0
    with open(dst_label_file, mode="w") as f:
        f.write(label_txt)
        f.close()

In [15]:
test_folder = "data_images/test"

values = df_test[["filename", "center_x", "center_y", "bb_width", "bb_height"]].values
for fname,x,y,w,h in values:
    image_name = os.path.split(fname)[-1] # it will give N1.jpeg etc.
    txt_name = os.path.splitext(image_name)[0] # it will give only N1 (image name without any format)
    
    dst_image_path = os.path.join(test_folder, image_name) # destination name for copy process
    dst_label_file = os.path.join(test_folder, txt_name+".txt")
    
    # copy each image into the folder
    copy(fname, dst_image_path)

    # generate .txt file which has label informatation.
    label_txt = f"0 {x} {y} {w} {h}" # -> class id will be 0
    with open(dst_label_file, mode="w") as f:
        f.write(label_txt)
        f.close()